Esta cosa debería de tomar dos archivos, una sucesión de tiempos del GD y otra del CA, y checar coincidencias. Usemos Julia solamente y tratemos de paralelizar.

In [2]:
# using HDF5, JLD

In [3]:
ListaCA=readdir("./Ejemplo01/CA3/");
ListaGD=readdir("./Ejemplo01/GD/");

In [4]:
EntradasYValores=Dict{Any,Any}()
Randomizados=Dict{Any,Any}()

Dict{Any,Any} with 0 entries

In [5]:
ejemploCA=readdlm("./Ejemplo01/CA3/CA3_Ch44_64.dat");
ejemploGD=readdlm("./Ejemplo01/GD/GD_Ch43_06.dat");

In [6]:
cuantosCA=length(ejemploCA)
cuantosGD=length(ejemploGD)
hasta=35
desde=14
coinciden=0
lastcoin=1

1

In [7]:
function CoincidenGDaCA(GD::Array, CA::Array)
    cuantosCA=length(CA)
    cuantosGD=length(GD)
    #Unidades en frames, son 1/7022 de segundo
    desde=14
    hasta=35
    coinciden=0
    lastcoin=1
    for j in 1:cuantosGD
        for k in lastcoin:cuantosCA
            if (CA[k]-GD[j]<hasta) && (CA[k]-GD[j]>desde)
                coinciden+=1
                lastcoin=k+1
                break
            end
        end
    end
    return coinciden
end

CoincidenGDaCA (generic function with 1 method)

In [8]:
typeof(parteCA)

LoadError: parteCA not defined
while loading In[8], in expression starting on line 1

In [9]:
CoincidenciasCausales=Array(Int, 1,7)
Pupu=Int32[]
laregexp=r"([0-9]{2})_([0-9]{2})"
@time for k in ListaCA, j in ListaGD
    ejemploCA=readdlm("./Ejemplo01/CA3/$k")
    ejemploGD=readdlm("./Ejemplo01/GD/$j")  
    parapa=match(laregexp, k)
    perepe=match(laregexp,j)
    parteCA=int(parapa.captures)
    parteGD=int(perepe.captures)
    cuantos=CoincidenGDaCA(ejemploGD,ejemploCA)
    Pupu=[parteGD, parteCA, [cuantos, length(ejemploGD), length(ejemploCA)]]'
    CoincidenciasCausales=vcat(CoincidenciasCausales, Pupu)
end


elapsed time: 12.110889489 seconds (4894664900 bytes allocated, 16.95% gc time)


In [10]:
writedlm("CoincidenciasPseudoRealesPureJulia01.dat", CoincidenciasCausales)

In [11]:
RandomCoincide=zeros(Float32, 1,7)
Pupu=Float32[]
cuantasveces=1000
promcoincide=0
@time for k in ListaCA, j in ListaGD
    ejemploCA=readdlm("./Ejemplo01/CA3/$k")
    ejemploGD=readdlm("./Ejemplo01/GD/$j")
    parapa=match(laregexp,k)
    perepe=match(laregexp,j)
    parteCA=int(parapa.captures)
    parteGD=int(perepe.captures)
    GDrev=vec(ejemploGD)
    CArev=vec(ejemploCA)
    for t in 1:cuantasveces
        shuffle!(GDrev)
        shuffle!(CArev)
        promcoincidede+=CoincidenGDaCA(GDrev, CArev)
    end
    promcoincide/=cuantasveces
    Pupu=[parteGD, parteCA, [promcoincide, length(GDrev), length(CArev)]]'
    RandomCoincide=vcat(RandomCoincide, Pupu)
end
writedlm("RevueltasCoincidenPureJulia01.dat", RandomCoincide)

elapsed time: 4315.412618936 seconds (5461612380 bytes allocated, 0.05% gc time)
